In [1]:
import polars as pl
import pandas as pd
import time
import numpy as np
import requests
import os
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import lightgbm as lgb
import seaborn as sns
import matplotlib.pyplot as plt
from catboost import CatBoostRegressor
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.compose import TransformedTargetRegressor
from sklearn.preprocessing import FunctionTransformer
from tqdm import tqdm
pd.set_option('display.max_columns', None)  # Toutes les colonnes
pd.set_option('display.width', None)        # Pas de limite de largeur pour les lignes
pd.set_option('display.max_colwidth', None) # Pas de limite pour la largeur des cellules


In [ ]:
df_test = pl.read_csv('data_pop_density/dataframe_densite_amenities.csv')
df_test.tail(5)

# Récupérations des données

## Etape de scrapping (Facultative)

In [ ]:
!python script_scrapping_dvf.py

## Chargement dans un DataFrame

### Fonctions

In [4]:
def change_column_types(df):
    """
    Change les types de plusieurs colonnes dans un DataFrame Polars.

    Parameters:
        df (pl.DataFrame): Le DataFrame Polars.
        columns_dtypes (dict): Un dictionnaire où les clés sont les noms de colonnes
                               et les valeurs sont les nouveaux types (ex: pl.Int64, pl.Float64).

    Returns:
        pl.DataFrame: Un nouveau DataFrame avec les colonnes modifiées.
    """
    dict_type ={'id_mutation': pl.Utf8, 'date_mutation': pl.Date, 'numero_disposition': pl.Utf8, 'nature_mutation': pl.Utf8, 'valeur_fonciere': pl.Float64, 'adresse_numero' : pl.Utf8, 'adresse_suffixe': pl.Utf8, 'adresse_nom_voie': pl.Utf8, 'adresse_code_voie' : pl.Utf8, 'code_postal':pl.Utf8, 'code_commune':pl.Utf8, 'nom_commune':pl.Utf8, 'code_departement':pl.Utf8, 'ancien_code_commune':pl.Utf8, 'ancien_nom_commune':pl.Utf8, 'id_parcelle':pl.Utf8, 'ancien_id_parcelle':pl.Utf8, 'numero_volume':pl.Utf8, 'lot1_numero':pl.Utf8, 'lot1_surface_carrez':pl.Float64, 'lot2_numero':pl.Utf8, 'lot2_surface_carrez':pl.Float64, 'lot3_numero':pl.Utf8, 'lot3_surface_carrez':pl.Float64, 'lot4_numero':pl.Utf8, 'lot4_surface_carrez':pl.Float64, 'lot5_numero':pl.Utf8, 'lot5_surface_carrez':pl.Float64, 'nombre_lots':pl.Float64, 'code_type_local': pl.Utf8, 'type_local': pl.Utf8, 'surface_reelle_bati': pl.Float64, 'nombre_pieces_principales': pl.Float64, 'code_nature_culture':pl.Utf8, 'nature_culture':pl.Utf8, 'code_nature_culture_speciale':pl.Utf8, 'nature_culture_speciale':pl.Utf8, 'surface_terrain':pl.Float64, 'longitude':pl.Float64, 'latitude':pl.Float64}
    for column_name, column_type in dict_type.items():
        df = df.with_columns(pl.col(column_name).cast(column_type))
    return df

def data_loader(path, departements = [], annees = []):

    df = pl.DataFrame()
    if not annees:
        annees_list = os.listdir(path)
    else:
        annees_list =[str(annee) for annee in annees]
    for annee in annees_list:
        cur_year = os.path.join(path,annee)
        if not departements:
            departements_list = os.listdir(cur_year)
        else:
            departements_list = [f"{departement}.csv.gz" if departement>9 else f"0{departement}.csv.gz" for departement in departements]
        for departement in departements_list:
            file = os.path.join(cur_year,departement)
            temp_df = pl.read_csv(file,ignore_errors=True)
            temp_df = change_column_types(temp_df)
            df = pl.concat([df, temp_df])

    return df


## Chargement des données:

In [5]:
path = 'data_dvf'
df = data_loader(path,departements=[75,92,93,94])

In [ ]:
df = df.filter(
    pl.col("valeur_fonciere").is_not_null() &
    pl.col("longitude").is_not_null() &
    pl.col("latitude").is_not_null() &
    (pl.col("surface_reelle_bati").is_not_nan() | pl.col("surface_terrain").is_not_nan())
)
df.head()

## BASELINE : Prétraitement des données et premier modèle

In [7]:
data = df[['date_mutation','type_local','surface_reelle_bati','nombre_pieces_principales','nature_culture','surface_terrain','longitude','latitude','valeur_fonciere']]

# # Calculer la composante saisonnière
data = data.with_columns([
    pl.col("date_mutation").dt.month().map_batches(lambda x: np.sin(2 * np.pi * x / 12), return_dtype=pl.Float64).alias("sin_month"),
    pl.col("date_mutation").dt.month().map_batches(lambda x: np.cos(2 * np.pi * x / 12), return_dtype=pl.Float64).alias("cos_month"),
    pl.col("date_mutation").dt.year().alias('year')
])

In [ ]:
data.head(2)

In [ ]:
data.describe()

In [10]:
from sklearn.model_selection import train_test_split
data_test = data.filter(pl.col('valeur_fonciere')<3e6)
y = data_test['valeur_fonciere']
X = data_test.drop(['valeur_fonciere'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
categorical_columns_onehot = ['nature_culture','type_local'] # Columns that need OneHotEncoding
numerical_columns = X.select(pl.col(pl.Float64)).columns # Numerical columns
# Encoding and imputer Pipeline

def replace_none_with_nan(X):
    numerical_columns = X.select(pl.col(pl.Float64)).columns
    for col in numerical_columns:
        X = X.with_columns(
            pl.col(col).cast(pl.Float64).fill_null(np.nan).alias(col)
            )
    return X

onehot_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Impute les valeurs manquantes
    ('onehot', OneHotEncoder(handle_unknown='ignore'))     # One-Hot Encoding
])

numeric_pipeline = Pipeline(steps=[
    ('replace_null_to_nan', FunctionTransformer(replace_none_with_nan)),
    ('imputer', SimpleImputer(missing_values=np.nan,strategy='constant',fill_value=0)),  # Remplit les NaN avec 0
    ('scaler', StandardScaler())                 # Standardisation
])

# Encoding pipeline

column_transformer =  ColumnTransformer(
    transformers=[
        ('onehot', onehot_pipeline, categorical_columns_onehot),
        ('numeric', numeric_pipeline, numerical_columns)
    ]
)

In [ ]:
from sklearn import set_config

# Configuration pour afficher graphiquement
set_config(display="diagram")

# Afficher la pipeline dans un notebook
display(column_transformer)


In [ ]:
model = LinearRegression()

LR_pipeline = Pipeline([
    ('preprocess', column_transformer),
    ('linear_model',model)
])
display(LR_pipeline)

In [ ]:
LR_pipeline.fit(X_train,y_train)
y_pred = LR_pipeline.predict(X_test)

r2 =  r2_score(y_test,y_pred)
print(f' R² = {r2}')

In [ ]:
model = lgb.LGBMRegressor(random_state=42, n_estimators=100, learning_rate=0.1)

lgb_pipeline = Pipeline([
    ('preprocess', column_transformer),
    ('linear_model',model)
])
display(lgb_pipeline)

In [ ]:
lgb_pipeline.fit(X_train,y_train)
y_pred = lgb_pipeline.predict(X_test)

r2 =  r2_score(y_test,y_pred)
print(f' R² = {r2}')

In [ ]:
validation = pl.DataFrame()
validation = pl.DataFrame({
    'y_test': y_test,
    'y_pred': y_pred
})

# Tracer avec seaborn
plt.figure(figsize=(8, 6))
sns.regplot(x=y_pred, y=y_test, scatter_kws={'alpha':0.5}, line_kws={'color':'red'})
plt.xlabel("y_pred")
plt.ylabel("y_test")
plt.xlim(0,5e6)
plt.ylim(0,5e6)
plt.title("Comparaison entre y_test et y_pred")
plt.show()


In [ ]:
class YToNumpyTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, y):
        if isinstance(y, (pl.DataFrame, pl.Series)):
            y = y.to_numpy()
        return np.log(y)
    def inverse_transform(self, y):
        # Si aucune transformation inverse n'est nécessaire, renvoyez simplement y
        return np.exp(y)

class ToNumpyTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # Si X est un DataFrame ou une série Polars, le convertir en NumPy array
        if isinstance(X, pl.DataFrame):
            return X.to_numpy()
        elif isinstance(X, pl.Series):
            return X.to_numpy().reshape(-1, 1)
        return X

model = CatBoostRegressor(iterations=1000, depth=6, learning_rate=0.1, loss_function='RMSE')
catboost_pipeline = Pipeline([
    ('preprocess', column_transformer),
    ('to_numpy',ToNumpyTransformer()),
    ('catboost_model',model)
])



# Pipeline pour X
catboost_pipeline = Pipeline([
    ('preprocess', column_transformer),
    ('to_numpy', ToNumpyTransformer()),
    ('catboost_model', model)
])

# Pipeline pour y avec TransformedTargetRegressor
final_pipeline = TransformedTargetRegressor(
    regressor=catboost_pipeline,
    transformer=YToNumpyTransformer()
)
display(final_pipeline)

In [ ]:
final_pipeline.fit(X_train,y_train)
y_pred = final_pipeline.predict(X_test)

r2 =  r2_score(y_test,y_pred)
print(f' R² = {r2}')
mape = mean_absolute_percentage_error(y_test, y_pred) * 100
print(f'MAPE: {mape:.2f}%')
validation = pl.DataFrame()
validation = pl.DataFrame({
    'y_test': y_test,
    'y_pred': y_pred
})

# Tracer avec seaborn
plt.figure(figsize=(8, 6))
sns.regplot(x=y_pred, y=y_test, scatter_kws={'alpha':0.5}, line_kws={'color':'red'})
plt.xlabel("y_pred")
plt.ylabel("y_test")
plt.xlim(0,5e6)
plt.ylim(0,5e6)
plt.title("Comparaison entre y_test et y_pred")
plt.show()


In [ ]:
residuals = y_test - y_pred
plt.scatter(y_pred, residuals, alpha=0.5)
plt.xlim(0,0.4e7)
plt.ylim(-0.2e7,0.3e7)
plt.axhline(0, color='red', linestyle='--')
plt.xlabel('Valeurs prédites')
plt.ylabel('Résidus')
plt.title('Graphique des résidus')
plt.show()


Beaucoup d'hétéroscédasticité

## Modele plus puissant: Feature engineering

In [ ]:
path = 'data_dvf'
df = data_loader(path,departements=[75,92,93,94])
df.head()

In [22]:
df = df.filter(
    pl.col("valeur_fonciere").is_not_null() &
    pl.col("longitude").is_not_null() &
    pl.col("latitude").is_not_null() &
    (pl.col("surface_reelle_bati").is_not_nan() | pl.col("surface_terrain").is_not_nan())
)

data = df[['date_mutation','type_local','surface_reelle_bati','nombre_lots','lot1_surface_carrez','lot2_surface_carrez','lot3_surface_carrez','lot4_surface_carrez','lot5_surface_carrez','nombre_pieces_principales','nature_culture','surface_terrain','longitude','latitude','valeur_fonciere']]


data = data.with_columns([
    pl.col("date_mutation").dt.month().map_batches(lambda x: np.sin(2 * np.pi * x / 12), return_dtype=pl.Float64).alias("sin_month"),
    pl.col("date_mutation").dt.month().map_batches(lambda x: np.cos(2 * np.pi * x / 12), return_dtype=pl.Float64).alias("cos_month"),
    pl.col("date_mutation").dt.year().alias('year'),
    pl.col('lot1_surface_carrez').fill_null(value=0),
    pl.col('lot2_surface_carrez').fill_null(value=0),
    pl.col('lot3_surface_carrez').fill_null(value=0),
    pl.col('lot4_surface_carrez').fill_null(value=0),
    pl.col('lot5_surface_carrez').fill_null(value=0),
    pl.col('surface_reelle_bati').fill_null(value=0),
    pl.col('surface_terrain').fill_null(value=0)
])


In [ ]:
data = data.with_columns(
    (pl.col('lot1_surface_carrez')+pl.col('lot2_surface_carrez')+pl.col('lot3_surface_carrez')+pl.col('lot4_surface_carrez')+pl.col('lot5_surface_carrez')
).alias("total_surface_carrez"))

data = data.with_columns(
    pl.when(pl.col("total_surface_carrez") == 0)
            .then(pl.col("surface_reelle_bati"))
            .otherwise(pl.col("total_surface_carrez"))
            .alias("total_surface_carrez"),
    pl.when(pl.col("surface_reelle_bati") != 0)
            .then((pl.col("valeur_fonciere")/pl.col("surface_reelle_bati")).round(0))
            .otherwise((pl.col("valeur_fonciere")/pl.col("surface_terrain")).round(0))
            .alias("valeur_fonciere_m2")
)
data.head()

In [24]:
from scipy.spatial import cKDTree
def idw_predict_kdtree(data, lon_col="longitude", lat_col="latitude", value_col="valeur_fonciere_m2", power=2, k=10):
    """
    Prédit les valeurs d'un DataFrame en utilisant l'Interpolation Inverse Distance Weighting (IDW) optimisée avec un KD-Tree.

    Paramètres :
        data : pl.DataFrame, DataFrame contenant les colonnes latitude, longitude et valeur foncière.
        lat_col : str, nom de la colonne latitude.
        lon_col : str, nom de la colonne longitude.
        value_col : str, nom de la colonne des valeurs à prédire.
        power : float, puissance de pondération (typiquement 2).
        k : int, nombre de voisins à considérer pour chaque point.

    Retourne :
        pl.Series, colonne des valeurs prédites pour chaque point.
    """
    # Extraire les coordonnées et les valeurs
    coordinates = data.select([lat_col, lon_col]).to_numpy()
    values = data[value_col].to_numpy()

    # Construire le KD-Tree
    tree = cKDTree(coordinates)

    predicted_values = []
    for i, point in enumerate(coordinates):
        # Trouver les k voisins les plus proches
        distances, indices = tree.query(point, k=k + 1)  # k+1 car le point lui-même est inclus

        # Exclure le point lui-même (distance 0)
        mask = distances > 0
        distances = distances[mask]
        indices = indices[mask]

        # Si aucun voisin valide n'est trouvé
        if len(distances) == 0:
            predicted_values.append(values[i])  # Retourner la valeur du point lui-même
            continue

        # Calculer les poids en fonction des distances
        weights = 1 / (distances ** power)

        # Calculer la valeur interpolée
        interpolated_value = np.round(np.sum(weights * values[indices]) / np.sum(weights),-1)
        predicted_values.append(interpolated_value)

    return pl.Series(predicted_values).alias(f"{value_col}_predite_par_le_quartier")


In [ ]:
data = data.with_columns(
    idw_predict_kdtree(data, lon_col="longitude",lat_col="latitude", value_col="valeur_fonciere_m2", k=100),
)
data.head()

In [ ]:
df_popd = pd.read_csv('data_pop_density/dataframe_densite&amenities_radius=500.csv')
df_popd.drop(columns='Unnamed: 0',inplace=True)
df_popd

In [ ]:
import numpy as np
import polars as pl
from scipy.spatial import cKDTree  # Importer cKDTree pour une recherche efficace des plus proches voisins

# Fonction pour trouver la densité la plus proche en utilisant cKDTree
def find_nearest_square_data(data, df_popd):
    """
    Trouve la densité de population du point le plus proche pour chaque point dans un DataFrame de données
    géographiques en utilisant un arbre k-d (cKDTree).

    Cette fonction cherche, pour chaque point dans `data`, le point le plus proche dans `df_popd`
    (en fonction de la distance géographique) et retourne la densité associée à ce point.

    Parameters:
    - data (polars.DataFrame): Le DataFrame contenant les coordonnées géographiques des points
      pour lesquels la densité doit être calculée. Il doit contenir les colonnes 'latitude' et 'longitude'.
    - df_popd (polars.DataFrame): Le DataFrame contenant les coordonnées géographiques des points
      de population et la densité associée. Il doit contenir les colonnes 'lat', 'lon' et 'densite'.

    Returns:
    - numpy.ndarray: Un tableau contenant les densités associées aux plus proches voisins pour
      chaque point de `data`.
    """
    latitudes_data = data['latitude'].to_numpy()
    longitudes_data = data['longitude'].to_numpy()

    latitudes_popd = df_popd['lat'].to_numpy()
    longitudes_popd = df_popd['lon'].to_numpy()
    densities_popd = df_popd['densite'].to_numpy()

    # Créer un cKDTree pour une recherche rapide des plus proches voisins
    tree = cKDTree(np.vstack((longitudes_popd, latitudes_popd)).T)

    # Chercher les voisins les plus proches pour chaque point de `data`
    distances, indices = tree.query(np.vstack((longitudes_data, latitudes_data)).T, k=4)

    # Récupérer les densités associées aux plus proches voisins
    nearest_densities = np.mean(densities_popd[indices], axis=1)

    return nearest_densities

# Appliquer la fonction pour ajouter la colonne 'nearest_density' au DataFrame
nearest_densities = find_nearest_square_data(data, df_popd)

# Ajouter la colonne 'nearest_density' à ton DataFrame Polars
data = data.with_columns(
    pl.Series('nearest_density', nearest_densities)
)

# Afficher le DataFrame résultant
data.head()


In [28]:
from scipy.spatial import cKDTree
import numpy as np
import polars as pl

def find_weighted_poi_counts(data, df_grid, poi_columns):
    """
    Calcule une moyenne pondérée des colonnes POIs des quatre voisins les plus proches
    pour chaque point du DataFrame `data` en utilisant un arbre k-d (cKDTree).

    Parameters:
    - data (polars.DataFrame): DataFrame contenant les coordonnées des points pour lesquels
      les POIs pondérés doivent être calculés. Il doit contenir les colonnes 'latitude' et 'longitude'.
    - df_grid (polars.DataFrame): DataFrame contenant le quadrillage avec coordonnées et colonnes POIs.
      Il doit contenir les colonnes 'lat', 'lon' et les colonnes spécifiées dans `poi_columns`.
    - poi_columns (list): Liste des noms des colonnes POIs dans `df_grid` à inclure dans les calculs.

    Returns:
    - polars.DataFrame: DataFrame enrichi avec les colonnes pondérées des POIs.
    """
    # Extraire les coordonnées des deux ensembles
    latitudes_data = data['latitude'].to_numpy()
    longitudes_data = data['longitude'].to_numpy()

    latitudes_grid = df_grid['lat'].to_numpy()
    longitudes_grid = df_grid['lon'].to_numpy()

    # Créer un cKDTree pour une recherche rapide
    tree = cKDTree(np.vstack((longitudes_grid, latitudes_grid)).T)

    # Chercher les 4 voisins les plus proches pour chaque point
    distances, indices = tree.query(np.vstack((longitudes_data, latitudes_data)).T, k=4)

    # Calculer les poids en fonction de l'inverse des distances
    weights = 1 / np.where(distances == 0, 1e-10, distances)  # Évite la division par zéro
    normalized_weights = weights / weights.sum(axis=1, keepdims=True)

    # Calculer les moyennes pondérées pour chaque colonne POI
    weighted_poi_data = {}
    for col in poi_columns:
        poi_values = df_grid[col].to_numpy()
        # Récupérer les valeurs des voisins pour cette colonne
        neighbors_poi = poi_values[indices]
        # Calculer la moyenne pondérée
        weighted_poi_data[f"{col}_weighted"] = np.floor((neighbors_poi * normalized_weights).sum(axis=1))

    # Ajouter les colonnes pondérées au DataFrame d'entrée
    return data.with_columns([
        pl.Series(name, values) for name, values in weighted_poi_data.items()
    ])


In [ ]:
# Colonnes POIs à inclure dans les calculs
poi_columns = ['transport_pois', 'education_pois','health_pois', 'food_pois', 'shopping_pois', 'park_pois',	'entertainment_pois', 'cultural_pois']

# Calculer les moyennes pondérées
data_expanded = find_weighted_poi_counts(data,df_popd, poi_columns)

data_expanded


In [ ]:
import seaborn as sns

# Conversion en pandas
df_pandas = data_expanded.to_pandas()

# Tracer avec Seaborn
sns.scatterplot(data=df_pandas, x="valeur_fonciere_m2", y="valeur_fonciere_m2_predite_par_le_quartier")
plt.plot([df_pandas["valeur_fonciere_m2"].min(), df_pandas["valeur_fonciere_m2"].max()],
         [df_pandas["valeur_fonciere_m2"].min(), df_pandas["valeur_fonciere_m2"].max()],
         color='red', linestyle='--', label="y=x Line")
plt.title("Scatter plot of valeur_fonciere_m2 vs valeur_fonciere_m2_predite_par_le_quartier")
plt.xlim((0,0.5e8))
plt.ylim((0,0.5e8))
plt.show()

In [ ]:
df_pandas['valeur_fonciere_m2_log']=df_pandas['valeur_fonciere_m2'].apply(lambda X: np.log10(X))


In [ ]:
sns.histplot(data=df_pandas,x='valeur_fonciere_m2_log',bins=150)

In [ ]:
df_pandas.columns

In [ ]:
mask = (df_pandas['valeur_fonciere_m2_log']<5.5) & (df_pandas['total_surface_carrez']+df_pandas['surface_terrain']>1) & (df_pandas["nombre_pieces_principales"].notnull())
test_df =df_pandas[mask]
test_df['valeur_fonciere_m2_predite_par_le_quartier_log']=test_df['valeur_fonciere_m2_predite_par_le_quartier'].apply(lambda X: np.log10(X))
sns.histplot(data=test_df,x='valeur_fonciere_m2_predite_par_le_quartier_log')

In [ ]:
test_df.sort_values(by='surface_terrain')

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Liste des colonnes à tracer
columns_to_plot = [
    'valeur_fonciere_m2_predite_par_le_quartier_log',
    'type_local',
    'total_surface_carrez'
]

# Nombre de colonnes pour l'affichage des subplots
n_cols = len(columns_to_plot)

# Configurer la figure avec plusieurs sous-graphiques
fig, axes = plt.subplots(1, n_cols, figsize=(5 * n_cols, 5))

# Tracer chaque histogramme
for ax, column in zip(axes, columns_to_plot):
    sns.histplot(data=test_df, x=column, ax=ax, color='skyblue')
    ax.set_title(column)

# Ajuster les espacements
plt.tight_layout()
plt.show()


In [ ]:
import plotly.express as px

# Exemple de DataFrame pandas
df_pandas_sampled = df_pandas.sample(10000)  # Prendre un échantillon de 10 000 points

# Création de la carte avec Plotly Express
fig = px.scatter_mapbox(
    df_pandas_sampled,
    lat="latitude",  # Colonne pour les latitudes
    lon="longitude",  # Colonne pour les longitudes
    size="valeur_fonciere_m2",  # Taille des cercles basée sur la valeur/m²
    color="valeur_fonciere_m2",  # Couleur des cercles basée sur la valeur/m²
    color_continuous_scale=px.colors.sequential.Blues,  # Palette de couleurs
    size_max=15,  # Taille maximale des cercles
    zoom=13,  # Niveau de zoom initial
    mapbox_style="carto-positron",  # Style de carte
    title="Carte des valeurs foncières au m²"
)

# Ajouter des popups avec les informations
fig.update_traces(
    hovertemplate="<b>Prix/m²</b>: %{marker.size} €<br><b>Latitude</b>: %{lat}<br><b>Longitude</b>: %{lon}"
)

# Afficher la carte
fig.show()
